In [1]:
import sys
import os
import shutil
import glob

# check if GEE is already imported to avoid requesting authenticatiation multiple times
modulename = 'PlotsManager'
if modulename not in sys.modules: 
   # import GEE and Authenticate, token or log in will be asked from web browser
   %run PlotsManager.ipynb
else:
   print('PlotsManager already imported')
   # google earth engine already imported and authenticated


In [2]:
 

class MergingLib:
    #  @param[in] nameOfCSVFolderDir the directory that contains all the exported
    #  .csv files. You need to download and extract this folder from Google Drive
    #  @param[in] fieldDataWithIdentifiers after each run, this is found at the 
    #  same directory as PlotToSat .ipynb files. It is the field data with an extra
    #  column named "indexField". This column saves some identifiers used to merge
    #  the field data with the exported EO spectral temporal signatures
    def __init__(self,nameOfCSVFolderDir,fieldDataWithIdentifiers):
        self.plotsStd = PlotsManager(fieldDataWithIdentifiers)
        self.plotsMean = PlotsManager(fieldDataWithIdentifiers)
        self.nameOfCSVFolderDir = nameOfCSVFolderDir

        self.ResDir  = os.path.join(self.nameOfCSVFolderDir,"MergedCsvs")
        if os.path.isdir(self.ResDir):
            print ("MergedCsvs Exist. Deleting all of its content!")
            shutil.rmtree(self.ResDir)
        os.mkdir(self.ResDir)
        self.ListS1Mean=[]
        self.ListS2Mean=[]
        self.ListS1StdD=[]
        self.ListS2StdD=[]

   

        for file1 in glob.glob(nameOfCSVFolderDir+"/*.csv"):
            S1Mean="S1_mean.csv"
            S2Mean="S2_mean.csv"
            S1StdD="S1_stdD.csv"
            S2StdD="S2_stdD.csv"
            if(len(file1)>11):
                if   (file1[len(file1)-11:len(file1)]==S1Mean):
                    self.ListS1Mean=self.ListS1Mean+[file1]
                elif (file1[len(file1)-11:len(file1)]==S2Mean):
                    self.ListS2Mean=self.ListS2Mean+[file1]
                elif (file1[len(file1)-11:len(file1)]==S1StdD):
                    self.ListS1StdD=self.ListS1StdD+[file1]
                elif (file1[len(file1)-11:len(file1)]==S2StdD):
                    self.ListS2StdD=self.ListS2StdD+[file1]
                elif (file1[len(file1)-11:len(file1)]=="tifiers.csv"): #in case field data are added in the nameOfCSVFolderDir
                    print(file1 , " is suspected to be the field data file")
                else :
                    print("WARNING: ", file1, " is ignored since it is not recognised as an output of the system")



        print ("ListS1Mean",len(self.ListS1Mean),
               "     ListS2Mean ", len(self.ListS2Mean), 
               "     ListS1StdD ", len(self.ListS1StdD), 
               "     ListS2StdD ", len(self.ListS2StdD))


    ## @brief method that merges all the exported data from PlotToSat into 
    #  a single .csv file
    def mergeAll(self):
        firstItem = None
        if(self.ListS1Mean!=[]):
            firstItem=self.ListS1Mean[0]
        elif (self.ListS2Mean!=[]):
            firstItem=ListS2Mean[0]
        else :
            raise Exception ("ERROR: no data found. Both Sentinel-1 and Sentinel-2 lists are empty")
    
        for filename in self.ListS1Mean :
            self.plotsMean.addInfo(filename)

        for filename in self.ListS2Mean :
            self.plotsMean.addInfo(filename)

        for filename in self.ListS1StdD :
            self.plotsStd.addInfo(filename)

        for filename in self.ListS2StdD :
            self.plotsStd.addInfo(filename)
        
        #self.plotsMean.print()
 
        fileNames = firstItem[0:len(firstItem)-34] 
        head, fileNames = os.path.split(fileNames)
        outMean = self.ResDir+"/"+fileNames+"_mean.csv"
        outstdD = self.ResDir+"/"+fileNames+"_stdD.csv"

        self.plotsMean.exportTo(outMean)
        self.plotsStd.exportTo(outstdD)
         
        print("Results are stored in ", self.ResDir)
